In [4]:
#OpenAI Gym ile rastgele bir ortamda test yapmak
import gym #openai yüklemek için 
import random

In [5]:
env = gym.make('CartPole-v0')#Carthpole ortamına ulaşmak içimn make yöntemi kullanıldı
states = env.observation_space.shape[0]
actions = env.action_space.n

In [6]:
actions #sağ ve sol hareketleri  tanımlar

2

In [7]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()#ortam yaratma
        action = random.choice([0,1]) #rastgele bir adım (sağ veya sola hareket)
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))



Episode:1 Score:76.0
Episode:2 Score:41.0
Episode:3 Score:13.0
Episode:4 Score:18.0
Episode:5 Score:12.0
Episode:6 Score:21.0
Episode:7 Score:26.0
Episode:8 Score:12.0
Episode:9 Score:19.0
Episode:10 Score:32.0


In [8]:
#keras ile derin öğrenme modeli yaratma
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [9]:
#model tanımlamlanır
def build_model(states, actions): #girdi olarak durumları ve aksiyonları alır
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [10]:
model = build_model(states, actions)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 4)                 0         
_________________________________________________________________
dense (Dense)                (None, 24)                120       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Keras RL ile ajanı(agent) inşa etmek
from rl.agents import DQNAgent #ajanların bulunduğu keras kütüphanesi
from rl.policy import BoltzmannQPolicy #policy reinforcement learning kullanılıyor
from rl.memory import SequentialMemory #hafızaya almak için

In [13]:
def build_agent(model, actions): #eğitim aşaması
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [14]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 30:47 - reward: 1.0000

/home/ece/home/ece/anaconda3.1/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


    4/10000 [..............................] - ETA: 3:15 - reward: 1.0000 

/home/ece/home/ece/anaconda3.1/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 282s 28ms/step - reward: 1.0000
105 episodes - episode_reward: 94.190 [11.000, 200.000] - loss: 3.804 - mae: 19.040 - mean_q: 38.501

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 243s 24ms/step - reward: 1.0000
52 episodes - episode_reward: 193.096 [162.000, 200.000] - loss: 6.904 - mae: 40.212 - mean_q: 81.128

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 297s 30ms/step - reward: 1.0000
50 episodes - episode_reward: 197.940 [170.000, 200.000] - loss: 5.785 - mae: 43.568 - mean_q: 87.768

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 299s 30ms/step - reward: 1.0000
51 episodes - episode_reward: 195.804 [150.000, 200.000] - loss: 5.270 - mae: 41.070 - mean_q: 82.564

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 239s 24ms/step - reward: 1.0000
done, took 1360.483 seconds


In [15]:
scores = dqn.test(env, nb_episodes=100, visualize=False) #test aşaması
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [16]:
_ = dqn.test(env, nb_episodes=15, visualize=True) #görselleştirme

Testing for 15 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200


In [17]:
#ajanın hafızasına yapılan örneği kaydetmek 
#buradaki amaç daha sonra buna benzer bir örnek yapıldığında ajanın önceki verilere erişim sağlamasına imkan vermek
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [18]:
del model
del dqn
del env

In [19]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])


In [20]:
dqn.load_weights('dqn_weights.h5f')

In [21]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
